## Question Generation

In [2]:
# collect all paragraphs
path = '../data/cast19/test_paragraphs.tsv'
qp_ids = {}
paras = {}
with open(path, encoding='utf-8') as f:
    for i, line in enumerate(f):
        q_id, para_id, para_text = line.rstrip().split('\t')
        if q_id not in qp_ids:
            qp_ids[q_id] = []
        qp_ids[q_id].append(para_id)
        if para_id not in paras:
            paras[para_id] = para_text
print(len(paras))
print(qp_ids)

6485
{'31_1': ['CAR_1463f964653c5c9f614a0a88d26b175e4a8120f1', 'CAR_172e16e89ea3d5546e53384a27c3be299bcfe968', 'CAR_2174ad0aa50712ff24035c23f59a3c2b43267650', 'CAR_25a576af9caa6422f55c2acf945dc79b423fb41e', 'CAR_2dc597ac2fc10917a752552bc335e6ac1aedc3f0', 'CAR_3249e5618575a849152c02b05f4fda924f10326f', 'CAR_393cb4e18a9d30018e843c4d37c564272ec5fa6f', 'CAR_40c64256e988c8103550008f4e9b7ce436d9536d', 'CAR_41ab200340a8fcae748b366ff0b0506e6b87cefa', 'CAR_41b7dce4f8a72ee34d78c2b5c363272a54997f27', 'CAR_55f68f0a33f49f6015035f70d8685e293389c9d6', 'CAR_5b50a00d9e1bd1d4ac3a973a656c1b20ab96ec1e', 'CAR_683d2822b14dca56ef1ffd93c997258c3b2cc083', 'CAR_71fe664149cf06de1fcbb55dfb239f48687753c0', 'CAR_7b85fdb20fa85b01050fc49656cbc548d231defe', 'CAR_87d4034ad40c274e7b3a40ab7cfe5aadc2fd214c', 'CAR_8b8521b6835068974cc26ff1dfb3d76f84b7a504', 'CAR_bc285835e2ded639fa3ca767f43c8ed4bbbbdfe6', 'CAR_d37891cd657001a45ae65a1864b6af96a9b4543c', 'CAR_dbeb8cd6edbddddd97b665073523548470dffe44', 'CAR_eed3d1eb3efc8660a010

In [3]:
# collect all GS questions: independent
path_qs = '../data/cast19/human_questions.txt'

gs_qs = {}
with open(path_qs, 'r') as f:
    next(f)
    for line in f:
        q_id, q = line.split('\t')
#         print(q_id)
        gs_qs[q_id] = (q.strip('\n'))
print(len(gs_qs))
print(gs_qs['1_11'])

748
What is the fastest way to become a nurse practitioner NP?


In [4]:
# test rouge scorer
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

scores = scorer.score("what is fsa health plan .", "what is health care fsa plan .")  # reference predicted
print(scores)

{'rouge1': Score(precision=0.8333333333333334, recall=1.0, fmeasure=0.9090909090909091), 'rougeL': Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)}


# Global Context

Global context means sequences of questions from the same conversation. Incorporate global awareness when selecting next questions with respect to the previously selected questions.

In [5]:
# group question ids into sequences within the same conversation
from collections import defaultdict

question_sequences = defaultdict(list)
for q_id in qp_ids:
    conversation_id = q_id.split('_')[0]
    question_sequences[conversation_id].append(q_id)
#     print(gs_qs[q_id])
print(question_sequences)

defaultdict(<class 'list'>, {'31': ['31_1', '31_2', '31_3', '31_4', '31_5', '31_6', '31_7', '31_8', '31_9'], '32': ['32_1', '32_2', '32_3', '32_4', '32_5', '32_6', '32_7', '32_8', '32_9', '32_10', '32_11'], '33': ['33_1', '33_2', '33_3', '33_4', '33_5', '33_6', '33_7', '33_8'], '34': ['34_1', '34_2', '34_3', '34_4', '34_5', '34_6', '34_7', '34_8'], '37': ['37_1', '37_2', '37_3', '37_4', '37_5', '37_6', '37_7', '37_8'], '40': ['40_1', '40_2', '40_3', '40_4', '40_5', '40_6', '40_7', '40_8'], '49': ['49_1', '49_2', '49_3', '49_4', '49_5', '49_6', '49_7', '49_8'], '50': ['50_1', '50_2', '50_3', '50_4', '50_5', '50_6', '50_7', '50_8'], '54': ['54_1', '54_2', '54_3', '54_4', '54_5', '54_6', '54_7', '54_8', '54_9'], '56': ['56_1', '56_2', '56_3', '56_4', '56_5', '56_6', '56_7', '56_8'], '58': ['58_1', '58_2', '58_3', '58_4', '58_5', '58_6', '58_7', '58_8'], '59': ['59_1', '59_2', '59_3', '59_4', '59_5', '59_6', '59_7', '59_8'], '61': ['61_1', '61_2', '61_3', '61_4', '61_5', '61_6', '61_7', '6

In [10]:
# load generated questions into a dictionary
import os
import json

generated_questions = defaultdict(list)
questions_specificity = defaultdict(list)

squash_path = '/home/svakule/squash-generation/squash/temp/trec_cast19'

sim_threshold = 0.8  # rouge similarity threshold for duplicate questions

for question_id in os.listdir(squash_path):
    print(question_id)
    try:
        with open(squash_path+'/%s/generated_questions.json' % question_id) as f:
            paragraphs = json.load(f)['data'][0]['paragraphs']
            # group all generated questions to reference questions
            for i, p in enumerate(paragraphs):
                qs, ss = [], []
                for qa in p['qas']:
#                     print(qa['class'])
                    # do not keep duplicate questions for the same paragraph
                    q = qa['question']#.lower()
                    if q not in qs:
                        # check similarity to existing questions
                        duplicate = False
                        for q_ref in qs:
        #                     print(q_ref)
                            scores = scorer.score(q_ref, q)
                            rouge1f = scores['rouge1'].fmeasure
                            if rouge1f > sim_threshold:
                                duplicate = True
                                break
        #                     print(rouge1f)
                        if not duplicate:
                            qs.append(q)
                            ss.append(qa['class'])
                generated_questions[question_id].extend(qs)
                questions_specificity[question_id].extend(ss)
#                 print(len(qs))
#         print(len(generated_questions[question_id]))
#         break
    except:
        pass

print(len(generated_questions))       

67_6
79_4
78_3
58_4
59_3
75_2
56_6
32_7
75_8
49_3
69_4
68_9
31_4
54_2
77_6
68_3
54_8
37_2
37_8
34_1
50_7
34_6
37_5
69_3
54_5
31_3
77_1
68_4
69_9
31_9
75_5
56_1
33_7
49_4
58_3
59_4
67_1
79_3
79_9
78_4
61_7
32_11
40_7
61_3
40_3
58_7
79_7
67_5
77_5
68_11
31_7
54_1
69_7
32_4
33_3
56_5
75_1
34_2
34_8
50_3
37_1
67_10
37_6
34_5
50_4
49_7
32_3
56_8
32_9
56_2
33_4
75_6
68_7
77_8
77_2
54_6
67_8
78_7
67_2
59_7
40_4
61_4
59_5
58_8
58_2
79_8
78_5
79_2
61_6
32_10
40_6
34_7
50_6
37_4
68_5
69_8
31_8
69_2
54_4
31_2
49_5
32_1
33_6
75_4
49_2
32_6
33_1
56_7
75_3
49_8
68_2
54_9
77_7
69_5
68_8
31_5
54_3
37_3
50_1
40_1
61_1
78_2
78_8
79_5
67_7
59_2
58_5
59_8
67_3
79_1
78_6
67_9
58_1
59_6
40_5
61_5
37_7
50_5
34_4
32_8
56_3
33_5
32_2
49_6
54_7
31_1
69_1
77_3
68_6
31_6
68_10
69_6
77_4
68_1
33_2
56_4
33_8
32_5
49_1
50_2
34_3
50_8
67_11
61_2
61_8
40_2
40_8
58_6
59_1
67_4
79_6
69_10
78_1
165


In [11]:
# question selection with embedding based similarity
from sklearn.metrics.pairwise import cosine_similarity

# load best model evaluated on STSbenchmark fine-tuned on semantic textual similarity task using AllNLI and STSbenchmark
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

# evaluate squash baseline wrt question sequences
import numpy as np

mean_scores = []

for sequence_id, q_ids in question_sequences.items():
#     if sequence_id != '40':
#         continue
    print(sequence_id)
    
    # rouge scores per conversation
    scores = []
    # interate over questions in a sequence
    generated_sequence = []
    for q_id in q_ids:
        
        # get reference question per id
        q_ref = gs_qs[q_id]
#         print("Reference:", q_ref)
        
        # get generated questions per id
        q_gens = generated_questions[q_id]
        q_specs = questions_specificity[q_id]
        if q_gens:
            # encode all questions as vectors (embed)
            q_vectors = model.encode(q_gens)
    #         print(len(q_vectors))

            # compare all questions to each other to select a centroid (most similar question)
            similarities_matrix = cosine_similarity(q_vectors, q_vectors)
            sum_similarities = sum(similarities_matrix).tolist()  # aggregate scores across all questions

            while True:
                # calculate the most similar (representative) question (centroid)
                max_score = max(sum_similarities)
                max_index = np.argmax(sum_similarities)
                q = q_gens[max_index]
                # skip duplicate questions
                if q in generated_sequence:
                    del sum_similarities[max_index]
                    del q_gens[max_index]
                    continue
                break
#                 print("Centroid:", q)
            # report on question specificity
            s = q_specs[max_index]
            # calculate score of the selected question to the reference question
            score = scorer.score(q_ref, q)  # reference predicted
    #         print(score)
            # report sample
            print("%s -> %s %.2f" % (q_ref, q, score['rouge1'].fmeasure), s)

            # accumulate generated questions and scores
            generated_sequence.append(q)
            scores.append(score['rouge1'].fmeasure)
    
    if scores:
        mean_scores.append(np.mean(scores))
#     break
print(len(mean_scores))
print(np.mean(mean_scores))

31
What is throat cancer? -> What is the diagnosis of Throat cancer? 0.73 general
Is throat cancer treatable? -> What is the definition of a cancer? 0.36 specific
Tell me about lung cancer. -> What is the disease called that causes the lung cancer? 0.27 specific
What are the symptoms of lung cancer? -> What type of symptoms might occur in patients with lung cancer? 0.56 specific
Can lung cancer spread to the throat? -> What is a symptom of metastatic lung cancer? 0.27 general
What causes throat cancer? -> What is the relationship between smoking and cancer? 0.33 general
What is the first sign of throat cancer? -> What is a symptom of a throat cancer? 0.62 specific
Is throat cancer the same as esophageal cancer? -> What type of cancer can cause esophageal cancer? 0.38 specific
What's the difference in symptoms of throat cancer and esophageal cancer? -> What might be a symptom of throat cancer? 0.50 general
32
What are the different types of sharks? -> What type of shark are they? 0.77 s

What is modern evidence for Darwin's theory? -> What was the basis for Darwin's view of evolution? 0.44 specific
What is the impact of Darwin's theory on modern biology? -> what type of theory did Darwin's  Theory of Evolution have? 0.45 specific
Compare and contrast microevolution and macroevolution. -> What is the interpretation of macroevolution? 0.17 specific
What is the relationship of microevolution and macroevolution to speciation? -> How did Darwin's theory evolve? 0.00 general
58
What is a real-time database? -> What is the database component? 0.55 general
How does real-time database differ from traditional ones? -> What kind of database is that? 0.13 specific
What are the advantages of real-time processing in real-time database? -> What does it do? 0.12 general
What are examples of important real-time databases? -> What kind of database is it? 0.43 specific
What are important applications of real-time databases? -> What kind of application was the database application 0.50 sp

How is Herbert Spencer's work related to Comte? -> Who was the first thinker to study the social sciences? 0.11 specific
What is the functionalist theory? -> What theory does functionalism discuss? 0.40 specific
What is the main criticism of the functionalist theory? -> who came up with a theory? 0.13 specific
How does the functionalist theory compare to conflict theory? -> What does the theory discuss? 0.43 specific
What are modern examples of conflict theory? -> What was the conflict theory about? 0.46 general
20
0.3614692281405846


In [18]:
# question selection oracle
from sklearn.metrics.pairwise import cosine_similarity

# load best model evaluated on STSbenchmark fine-tuned on semantic textual similarity task using AllNLI and STSbenchmark
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

# evaluate squash baseline wrt question sequences
import numpy as np

mean_scores = []

for sequence_id, q_ids in question_sequences.items():
#     if sequence_id != '40':
#         continue
    print(sequence_id)
    
    # rouge scores per conversation
    scores = []
    # interate over questions in a sequence
    generated_sequence = []
    for q_id in q_ids:
        
        # get reference question per id
        q_ref = gs_qs[q_id]
#         print("Reference:", q_ref)
        
        # get generated questions per id
        q_gens = generated_questions[q_id]
        q_specs = questions_specificity[q_id]
        if q_gens:
            all_q_gens_scores = []
            for q in q_gens:
                _scores = scorer.score(q_ref, q)  # reference predicted
                rouge1f = _scores['rouge1'].fmeasure
        #         print(scores['rouge1'].fmeasure)
                all_q_gens_scores.append(_scores['rouge1'].fmeasure)
            
            while True:
                # calculate the most similar (representative) question (centroid)
                max_score = max(all_q_gens_scores)
                max_index = np.argmax(all_q_gens_scores)
                q = q_gens[max_index]
                
                # skip duplicate questions
                if q.lower() in generated_sequence:
                    del all_q_gens_scores[max_index]
                    del q_gens[max_index]
                    continue
                break
#                 print("Centroid:", q)
    
            # report on question specificity
            s = q_specs[max_index]
            # calculate score of the selected question to the reference question
            score = scorer.score(q_ref, q)  # reference predicted
    #         print(score)
            # report sample
            print("%s -> %s %.2f" % (q_ref, q, score['rouge1'].fmeasure), s)

            # accumulate generated questions and scores
            generated_sequence.append(q.lower())
            scores.append(score['rouge1'].fmeasure)
    
    if scores:
        mean_scores.append(np.mean(scores))
#     break
print(len(mean_scores))
print(np.mean(mean_scores))

31
What is throat cancer? -> What is Throat cancer? 1.00 general
Is throat cancer treatable? -> When is throat cancer diagnosed? 0.67 general
Tell me about lung cancer. -> What else can you tell me about lung cancer? 0.71 general
What are the symptoms of lung cancer? -> What are the symptoms of lung cancer? 1.00 general
Can lung cancer spread to the throat? -> Where can Throat cancer spread to? 0.77 specific
What causes throat cancer? -> What can cause throat cancer? 0.89 general
What is the first sign of throat cancer? -> What is the first sign of throat cancer? 1.00 specific
Is throat cancer the same as esophageal cancer? -> What is the esophageal cancer? 0.62 general
What's the difference in symptoms of throat cancer and esophageal cancer? -> What's the first symptom of Esophageal Cancer? 0.70 general
32
What are the different types of sharks? -> What are the different types of sharks? 1.00 general
Are sharks endangered?  If so, which species? -> How many shark species are endangere

What is modern evidence for Darwin's theory? -> What is Darwin's theory about? 0.71 general
What is the impact of Darwin's theory on modern biology? -> What is Darwin's theory of evolution? 0.67 general
Compare and contrast microevolution and macroevolution. -> what are macroevolution and microevolution? 0.55 general
What is the relationship of microevolution and macroevolution to speciation? -> What is the difference between macroevolution and microevolution? 0.67 general
58
What is a real-time database? -> what is a real time database 1.00 general
How does real-time database differ from traditional ones? -> How does a real time graphics system differ from traditional off-line rendering systems? 0.61 general
What are the advantages of real-time processing in real-time database? -> What are the main advantages of real time operating system 0.64 general
What are examples of important real-time databases? -> What are some examples of real-time databases? 0.88 general
What are important a

How does the functionalist theory compare to conflict theory? -> How many things does the theory of conflict theory claim? 0.63 specific
What are modern examples of conflict theory? -> What are the assumptions of modern conflict theory? 0.80 general
20
0.7046424009914773


In [ ]:
# compare question specificity sequence patterns for human vs generated questions
# question sequences -> keywords / specificity

In [ ]:
# use keyword-based or specificity-based templates to select questions